In [1]:
import sys

sys.path.insert(0, "..")

In [2]:
data_dir = "/mnt/work/Code/credit_card_fd/Multi-GNN/data/HI-Small_Trans.csv"

In [3]:
import pandas as pd

/tmp/ipykernel_12898/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df = pd.read_csv(data_dir)

In [18]:
# distict values of From Bank and To Bank
bank_id = set()
for i in df["From Bank"]:
    bank_id.add(i)
for i in df["To Bank"]:
    bank_id.add(i)

In [21]:
len(bank_id)

30470

In [6]:
df.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
count,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06
mean,4.573057e+04,6.574456e+04,5.988726e+06,4.509273e+06,1.019427e-03
std,8.176562e+04,8.409299e+04,1.037183e+09,8.697728e+08,3.191219e-02
min,1.000000e+00,1.000000e+00,1.000000e-06,1.000000e-06,0.000000e+00
25%,1.190000e+02,4.259000e+03,1.833700e+02,1.844800e+02,0.000000e+00
50%,9.679000e+03,2.156800e+04,1.411010e+03,1.414540e+03,0.000000e+00
75%,2.862800e+04,1.223320e+05,1.234627e+04,1.229784e+04,0.000000e+00
max,3.563030e+05,3.562940e+05,1.046302e+12,1.046302e+12,1.000000e+00


In [55]:
dfs = dict()
# iter though the data and create a dictionary of dataframes
for i in bank_id:
    dfs[i] = df[(df["From Bank"] == i) | (df["To Bank"] == i)]

In [59]:
for i in dfs:
    dfs[i].to_csv(f"../data/Small_HI_split/{i}.csv", index=False)

In [68]:
import numpy as np
import datatable as dt
from datetime import datetime
from datatable import f,join,sort
import sys
import os

def export(inPath, index):
    outPath = f"../data/Small_HI_split_trans/" + str(index) + ".csv"

    raw = dt.fread(inPath, columns = dt.str32)

    currency = dict()
    paymentFormat = dict()
    bankAcc = dict()
    account = dict()

    def get_dict_val(name, collection):
        if name in collection:
            val = collection[name]
        else:
            val = len(collection)
            collection[name] = val
        return val

    header = "EdgeID,from_id,to_id,Timestamp,\
    Amount Sent,Sent Currency,Amount Received,Received Currency,\
    Payment Format,Is Laundering\n"

    firstTs = -1

    with open(outPath, 'w') as writer:
        writer.write(header)
        for i in range(raw.nrows):
            datetime_object = datetime.strptime(raw[i,"Timestamp"], '%Y/%m/%d %H:%M')
            ts = datetime_object.timestamp()
            day = datetime_object.day
            month = datetime_object.month
            year = datetime_object.year
            hour = datetime_object.hour
            minute = datetime_object.minute

            if firstTs == -1:
                startTime = datetime(year, month, day)
                firstTs = startTime.timestamp() - 10

            ts = ts - firstTs

            cur1 = get_dict_val(raw[i,"Receiving Currency"], currency)
            cur2 = get_dict_val(raw[i,"Payment Currency"], currency)

            fmt = get_dict_val(raw[i,"Payment Format"], paymentFormat)

            fromAccIdStr = raw[i,"From Bank"] + raw[i,2]
            fromId = get_dict_val(fromAccIdStr, account)

            toAccIdStr = raw[i,"To Bank"] + raw[i,4]
            toId = get_dict_val(toAccIdStr, account)

            amountReceivedOrig = float(raw[i,"Amount Received"])
            amountPaidOrig = float(raw[i,"Amount Paid"])

            isl = int(raw[i,"Is Laundering"])

            line = '%d,%d,%d,%d,%f,%d,%f,%d,%d,%d\n' % \
                        (i,fromId,toId,ts,amountPaidOrig,cur2, amountReceivedOrig,cur1,fmt,isl)

            writer.write(line)

    formatted = dt.fread(outPath)
    formatted = formatted[:,:,sort(3)]

    formatted.to_csv(outPath)

In [70]:
for i in dfs:
    print(f"../data/Small_HI_split/{i}.csv")
    export(f"../data/Small_HI_split/{i}.csv", i)
    # break

../data/Small_HI_split/1.csv
../data/Small_HI_split/3.csv
../data/Small_HI_split/4.csv
../data/Small_HI_split/5.csv
../data/Small_HI_split/6.csv
../data/Small_HI_split/7.csv
../data/Small_HI_split/8.csv
../data/Small_HI_split/9.csv
../data/Small_HI_split/10.csv
../data/Small_HI_split/11.csv
../data/Small_HI_split/12.csv
../data/Small_HI_split/14.csv
../data/Small_HI_split/15.csv
../data/Small_HI_split/16.csv
../data/Small_HI_split/131086.csv
../data/Small_HI_split/19.csv
../data/Small_HI_split/20.csv
../data/Small_HI_split/21.csv
../data/Small_HI_split/22.csv
../data/Small_HI_split/23.csv
../data/Small_HI_split/24.csv
../data/Small_HI_split/26.csv
../data/Small_HI_split/27.csv
../data/Small_HI_split/28.csv
../data/Small_HI_split/29.csv
../data/Small_HI_split/70.csv
../data/Small_HI_split/71.csv
../data/Small_HI_split/131167.csv
../data/Small_HI_split/110.csv
../data/Small_HI_split/112.csv
../data/Small_HI_split/113.csv
../data/Small_HI_split/115.csv
../data/Small_HI_split/116.csv
../da